In [1]:
import numpy as np # linear algebra
import pandas as pd
# from ImplicitALS import ImplicitALS as als

In [ ]:
# model = als()
customers = pd.read_csv('../data/customers.csv')


In [2]:
articles = pd.read_csv('../data/articles.csv')


In [ ]:
transactions = pd.read_csv('../data/transactions_train.csv')

In [ ]:
# count = pd.read_parquet('../data/new-art-count.parquet')
# count

In [ ]:
# customers
# articles
transactions

In [ ]:
# df = transactions.groupby(['article_id','customer_id']).count()
# df.rename(columns={"t_dat":"count"},inplace=True)


In [ ]:
# df = df["count"].reset_index()
# df

In [ ]:
# atransactions = pd.merge(transactions,df )

In [3]:
from sklearn.preprocessing import LabelEncoder
ALL_ITEMS = articles['article_id'].unique().tolist()
leItem = LabelEncoder().fit(ALL_ITEMS)

In [ ]:
from sklearn.preprocessing import LabelEncoder

ALL_USERS = customers['customer_id'].unique().tolist()
ALL_ITEMS = articles['article_id'].unique().tolist()

leUser = LabelEncoder().fit(ALL_USERS)
transactions['user_id'] = leUser.transform(transactions['customer_id'])
leItem = LabelEncoder().fit(ALL_ITEMS)
transactions['item_id'] = leItem.transform(transactions['article_id'])
transactions

In [ ]:
from scipy.sparse import coo_matrix

col = transactions['user_id'].values
row = transactions['item_id'].values
one = np.ones(transactions.shape[0])
# one = df["count"].to_xarray
# one = atransactions['count'].values

coo = coo_matrix((one, (row, col)), shape=(len(ALL_ITEMS), len(ALL_USERS)))

In [ ]:
# coo
print( coo )

In [ ]:
csr = coo.tocsr()
csr

In [ ]:
csr.toarray

In [ ]:
print(csr)

In [5]:
import scipy.sparse
# scipy.sparse.save_npz('../data/sparse_matrix.npz', csr)
sparse_matrix = scipy.sparse.load_npz('../data/sparse_matrix.npz') ###### CSR  #####

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder

In [ ]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
# model_knn.fit(csr)
model_knn.fit(sparse_matrix)

In [6]:
from pickle import dump, load 
file = "../data/newKnn.pickle"
# dump(model_knn,open(file,"wb"))
K = load(open(file,"rb"))

In [7]:
query_artcile = 519936007 # input article ID
# query_artcile = 891322004
# leItem.inverse_transform([query_index])
index = leItem.transform([query_artcile])
# print("Choosen article is: ",article_table.index[query_index])

In [ ]:
# csr[:, 298].toarray()

In [8]:
# distances, indices = model_knn.kneighbors(csr[index,:].toarray().reshape(1,-1), n_neighbors = 6)
# distances, indices = K.kneighbors(csr[index,:].toarray().reshape(1,-1), n_neighbors = 6)
distances, indices = K.kneighbors(sparse_matrix[index,:].toarray().reshape(1,-1), n_neighbors = 6)

In [9]:
article = []
distance = []

for i in range(0, len(distances.flatten())):
    if i != 0:
        # article.append(indices.flatten()[i])
        article.append(leItem.inverse_transform([indices.flatten()[i]]))
        distance.append(distances.flatten()[i]) 
           

m=pd.Series(article,name='article')
d=pd.Series(distance,name='distance')
recommend = pd.concat([m,d], axis=1)
recommend = recommend.sort_values('distance',ascending=False)

# print('Recommendations for {0}:\n'.format(article_table.index[query_index]))
for i in range(0,recommend.shape[0]):
    print('{0}: {1}, with distance of {2}'.format(i, recommend["article"].iloc[i], recommend["distance"].iloc[i]))

0: [881786003], with distance of 0.7278344730240913
1: [745166002], with distance of 0.6797436923898257
2: [617923003], with distance of 0.6666666666666665
3: [860365003], with distance of 0.6666666666666665
4: [686318007], with distance of 0.6348516283298892


In [ ]:
x = np.array([0,3,1,0])
y = np.array([0,3,1,0])
z = np.array([1,1,1,1])
coo_matrix((z , (x,y)),shape=(4,4)).toarray()

